# Clean and organising the raw data scraped from https://stats.ncaa.org/

In [1]:
import pandas as pd
import os
import sys
from pathlib import Path

# Clean Data

In [2]:
year = 2019
for root, dirs, files in os.walk(f'../../data/ncaa/raw/{year}/team_game_by_game/'):
    for f in files:
        df = pd.read_csv(Path(root).joinpath(f), header=1)
        df.drop(columns=["MP", "Attend", "BHE", "Unnamed: 20"], inplace=True)
        df.replace({'/':''}, regex=True, inplace=True)
        df.fillna(0, inplace=True)
        df[["Kills", "Errors", "Total Attacks", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr"]] = df[["Kills", "Errors", "Total Attacks", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr"]].astype(int)
        outpath = Path(root).parent.parent.parent.joinpath(f"processed/{year}/game_by_game_cleaned/")
        outpath.mkdir(parents=True, exist_ok=True)
        f = f[:f.find('(') - 1] + ".csv"
        df.to_csv(outpath.joinpath(f), index=False)

# Computing Moving Averages

## Simple Moving Average

In [3]:
window = 10
year = 2019

for root, dirs, files in os.walk(f'../../data/ncaa/processed/{year}/game_by_game_cleaned'):
    new_root = Path(root).parent.joinpath(f"game_by_game_{window}_sma")
    new_root.mkdir(parents=True, exist_ok=True)
    for f in files:
        df = pd.read_csv(Path(root).joinpath(f))
        features = ["Kills", "Errors", "Total Attacks", "Hit Pct", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr", "PTS"]
        df[features] = df[features].rolling(window, min_periods=1).mean()
        df.to_csv(new_root.joinpath(f), index=False)

## Cumulative Moving Average

In [4]:
year = 2019

for root, dirs, files in os.walk(f'../../data/ncaa/processed/{year}/game_by_game_cleaned'):
    new_root = Path(root).parent.joinpath("game_by_game_cma")
    new_root.mkdir(parents=True, exist_ok=True)
    for f in files:
        df = pd.read_csv(Path(root).joinpath(f))
        features = ["Kills", "Errors", "Total Attacks", "Hit Pct", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr", "PTS"]
        df[features] = df[features].expanding().mean()
        df.to_csv(new_root.joinpath(f), index=False)

## Exponential Moving Average

In [5]:
year = 2019
alpha = 0.2

for root, dirs, files in os.walk(f'../../data/ncaa/processed/{year}/game_by_game_cleaned'):
    new_root = Path(root).parent.joinpath(f"game_by_game_{alpha}_ewm")
    new_root.mkdir(parents=True, exist_ok=True)
    for f in files:
        df = pd.read_csv(Path(root).joinpath(f))
        features = ["Kills", "Errors", "Total Attacks", "Hit Pct", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr", "PTS"]
        df[features] = df[features].ewm(alpha=alpha).mean()
        df.to_csv(new_root.joinpath(f), index=False)

# Combine into single dataframe of matches

## Utility Function

In [6]:
def clean_name(name):
    if '@' in name:
        if name.index('@') == 0:
            return name[2:]
        else:
            return name[:name.index('@')-1]
    else:
        return name

features = ["Kills", "Errors", "Total Attacks", "Hit Pct", "Assists", "Aces", "SErr", "Digs", "RErr", "Block Solos", "Block Assists", "BErr", "PTS"]
combined_features = ["Date", "TeamA", "TeamB", "Result", "S", "Team A Kills", "Team A Errors", "Team A Total Attacks", "Team A Hit Pct", "Team A Assists", "Team A Aces", "Team A SErr", "Team A Digs", "Team A RErr", "Team A Block Solos", "Team A Block Assists", "Team A BErr", "Team A PTS", "Team B Kills", "Team B Errors", "Team B Total Attacks", "Team B Hit Pct", "Team B Assists", "Team B Aces", "Team B SErr", "Team B Digs", "Team B RErr", "Team B Block Solos", "Team B Block Assists", "Team B BErr", "Team B PTS"]


def combine(input_path, output_path):
    dfs = []
    team_names = []
    for root, dirs, files in os.walk(input_path):
        for f in files:
            team_names.append(f[:-4])
            dfs.append(pd.read_csv(Path(root).joinpath(f)))

    data = []

    err_a = 0
    err_b = 0

    for i, name in enumerate(team_names):
        df = dfs[i]
        for j, TeamA_row in df.iterrows(): 
            date = TeamA_row["Date"]
            TeamA = name
            TeamB = clean_name(TeamA_row["Opponent"])
            Result = 1 if TeamA_row["Result"][0] == 'W' else 0
            S = TeamA_row["S"]
            TeamA_stats = TeamA_row[features]
            try:
                TeamB_df = dfs[team_names.index(TeamB)]
            except:
                err_a += 1
                continue
            try:
                TeamB_row = TeamB_df[TeamB_df["Date"] == date][TeamB_df["Opponent"].str.contains(TeamA)].reset_index().loc[0]
            except:
                err_b += 1
                continue

            TeamB_stats = TeamB_row[features]
            data.append([date, TeamA, TeamB, Result, S, *TeamA_stats, *TeamB_stats])
        
    combined_df = pd.DataFrame(data, columns=combined_features)
    combined_df.to_csv(output_path, index=False)
    return combined_df, dict(err_a=err_a, err_b=err_b)

def prev_combine(input_path, output_path):
    dfs = []
    team_names = []
    for root, dirs, files in os.walk(input_path):
        for f in files:
            team_names.append(f[:-4])
            dfs.append(pd.read_csv(Path(root).joinpath(f)))

    data = []

    err_a = 0
    err_b = 0

    for i, name in enumerate(team_names):
        df = dfs[i]
        for j in range(len(df)):
            if j == 0:
                continue
            TeamA_row = df.loc[j-1]
            date = TeamA_row["Date"]
            TeamA = name
            TeamB = clean_name(TeamA_row["Opponent"])
            Result = 1 if TeamA_row["Result"][0] == 'W' else 0
            S = TeamA_row["S"]
            TeamA_stats = TeamA_row[features]
            try:
                TeamB_df = dfs[team_names.index(TeamB)]
            except:
                err_a += 1
                continue
            try:
                TeamB_row_index = TeamB_df[TeamB_df["Date"] == date][TeamB_df["Opponent"].str.contains(TeamA)].index[0]
                if TeamB_row_index == 0:
                    continue
                TeamB_row = TeamB_df.loc[TeamB_row_index-1]
            except:
                err_b += 1
                continue

            TeamB_stats = TeamB_row[features]
            data.append([date, TeamA, TeamB, Result, S, *TeamA_stats, *TeamB_stats])
        
    combined_df = pd.DataFrame(data, columns=combined_features)
    combined_df.to_csv(output_path, index=False)
    return combined_df, dict(err_a=err_a, err_b=err_b)



## Combine dataframe of math by match result without any averages

In [11]:
year = 2019
input_path = f'../../data/ncaa/processed/{year}/game_by_game_cleaned'
output_path = f'../../data/ncaa/processed/{year}/accumulated/matches_gathered.csv'

matches_gathered_df, info = combine(input_path, output_path)
print(info, len(matches_gathered_df))

{'err_a': 368, 'err_b': 59} 9536


## Combine dataframe for Simple Moving Average

In [12]:
year = 2019
window = 10
input_path = f'../../data/ncaa/processed/{year}/game_by_game_{window}_sma'
output_path = f'../../data/ncaa/processed/{year}/accumulated/{window}_sma.csv'

sma_df, info = prev_combine(input_path, output_path)
print(info, len(sma_df))

{'err_a': 361, 'err_b': 57} 8891


## Combine dataframe for Cumulative Moving Average

In [13]:
year = 2019
input_path = f'../../data/ncaa/processed/{year}/game_by_game_cma'
output_path = f'../../data/ncaa/processed/{year}/accumulated/cma.csv'

cma_df, info = prev_combine(input_path, output_path)
print(info, len(cma_df))

{'err_a': 361, 'err_b': 57} 8891


## Combine dataframe for Exponentially Moving Average

In [14]:
year = 2019
alpha = 0.2
input_path = f'../../data/ncaa/processed/{year}/game_by_game_{alpha}_ewm'
output_path = f'../../data/ncaa/processed/{year}/accumulated/{alpha}_ewm.csv'

ewm_df, info = prev_combine(input_path, output_path)
print(info, len(ewm_df))

{'err_a': 361, 'err_b': 57} 8891
